In [1]:
import mxnet
import gluonnlp as nlp
import pandas as pd

TRAIN_PATH = '/home/dongha/code/data/conversation/Training/final_Training.xlsx'

# Index(['번호', '연령', '성별', '상황키워드', '신체질환', '감정_대분류', '감정_소분류', '사람문장1',
#  '시스템응답1', '사람문장2', '시스템응답2', '사람문장3', '시스템응답3', '사람문장4', '시스템응답4'],
# dtype='object')
data = pd.read_excel(TRAIN_PATH)




In [2]:
word2num = {}
num2word = {}

for i in data['감정_소분류']:
  val = word2num.get(i)
  if val: continue
  tmp = len(word2num)
  word2num[i] = tmp
  num2word[tmp] = i

print(len(word2num), word2num)


58 {'신이 난': 3, '스트레스 받는': 1, '당황': 2, '안도': 3, '취약한': 4, '우울한': 5, '구역질 나는': 6, '좌절한': 7, '눈물이 나는': 8, '배신당한': 9, '남의 시선을 의식하는': 10, '죄책감의': 11, '당혹스러운': 12, '분노': 13, '두려운': 14, '낙담한': 15, '회의적인': 16, '마비된': 17, '혼란스러운': 18, '흥분': 19, '한심한': 20, '염세적인': 21, '안달하는': 22, '걱정스러운': 23, '조심스러운': 24, '충격 받은': 25, '상처': 26, '불안': 27, '노여워하는': 28, '기쁨': 29, '짜증내는': 30, '환멸을 느끼는': 31, '비통한': 32, '혐오스러운': 33, '억울한': 34, '만족스러운': 35, '부끄러운': 36, '버려진': 37, '고립된': 38, '괴로워하는': 39, '자신하는': 40, '외로운': 41, '방어적인': 42, '편안한': 43, '질투하는': 44, '감사하는': 45, '초조한': 46, '실망한': 47, '슬픔': 48, '신뢰하는': 49, '가난한, 불우한': 50, '후회되는': 51, '툴툴대는': 52, '성가신': 53, '열등감': 54, '희생된': 55, '느긋': 56, '악의적인': 57}


In [3]:
data_list = []

for q, label in zip(data['사람문장1'], data['감정_소분류']):
  tmp = []
  tmp.append(q)
  tmp.append(str(word2num[label]))

  data_list.append(tmp)

In [5]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])
print(len(data_list))

['아내가 드디어 출산하게 되어서 정말 신이 나.', '3']
['주변 사람들이 나에게 무능하다고 말해서 씁쓸해.', '5']
['엄마가 지금이라도 공부 말고 다른 길을 알아보자고 하셔서 슬퍼.', '15']
['오늘 면접을 보고 왔는데 대답을 잘 못 해서 나 자신한테 실망스러워.', '47']
['요즘 영어 수학 점수도 떨어지고 학원에 가면 스트레스만 받는 것 같아.', '49']
['아니 사람들은 왜 자기 일도 아니면서 참견하고 난리야?', '13']
['친구 관계가 너무 힘들어. 베푸는 만큼 돌아오지 않는 것 같아.', '1']
40879


In [6]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [7]:
print(len(dataset_train))
print(len(dataset_test))

30659
10220


In [8]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/dongha/code/nlp/.cache/kobert_v1.zip
using cached model. /home/dongha/code/nlp/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [9]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [10]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [11]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /home/dongha/code/nlp/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [12]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [13]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=58,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [14]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/home/dongha/anaconda3/envs/nlp/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [16]:
for e in range(50):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/home/dongha/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/480 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9085631370544434 train acc 0.421875
epoch 1 batch id 201 loss 2.181889295578003 train acc 0.5162468905472637
epoch 1 batch id 401 loss 2.003206968307495 train acc 0.5164432668329177
epoch 1 train acc 0.5169270833333334


/home/dongha/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/160 [00:00<?, ?it/s]

...

epoch 19 batch id 1 loss 0.03364159166812897 train acc 1.0
epoch 19 batch id 201 loss 0.07676191627979279 train acc 0.9717039800995025
epoch 19 batch id 401 loss 0.13740701973438263 train acc 0.9698020573566085
epoch 19 train acc 0.9694986979166667


  0%|          | 0/160 [00:00<?, ?it/s]

epoch 19 test acc 0.41209161931818183


  0%|          | 0/480 [00:00<?, ?it/s]

epoch 20 batch id 1 loss 0.10146326571702957 train acc 0.96875
epoch 20 batch id 201 loss 0.12864767014980316 train acc 0.9681281094527363
